In [1]:
import pyspark
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession

conf = SparkConf()
conf.set("spark.ui.enabled", "true")

spark = SparkSession.builder.config(conf=conf).master("local[*]").appName("test").getOrCreate()

23/06/18 08:28:05 WARN Utils: Your hostname, Deepaks-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.40 instead (on interface en0)
23/06/18 08:28:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/18 08:28:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/fhv_tripdata_2021-01.parquet

--2023-06-17 17:17:52--  https://d37ci6vzurychx.cloudfront.net/trip-data/fhv_tripdata_2021-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.244.115.202, 18.244.115.167, 18.244.115.107, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.244.115.202|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11886281 (11M) [binary/octet-stream]
Saving to: ‘fhv_tripdata_2021-01.parquet’

fhv_tripdata_2021-0 100%[===================>]  11.33M  10.5MB/s    in 1.1s    

2023-06-17 17:17:54 (10.5 MB/s) - ‘fhv_tripdata_2021-01.parquet’ saved [11886281/11886281]



In [7]:
!wc -l fhv_tripdata_2021-01.parquet

   32645 fhv_tripdata_2021-01.parquet


In [2]:
df = spark.read.option("header", "true").parquet("fhv_tripdata_2021-01.parquet")

In [71]:
df.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B00009|2021-01-01 00:27:00|2021-01-01 00:44:00|        null|        null|   null|                B00009|
|              B00009|2021-01-01 00:50:00|2021-01-01 01:07:00|        null|        null|   null|                B00009|
|              B00013|2021-01-01 00:01:00|2021-01-01 01:51:00|        null|        null|   null|                B00013|
|              B00037|2021-01-01 00:13:09|2021-01-01 00:21:26|        null|        72.0|   null|                B00037|
|              B00037|2021-01-01 00:38:31|2021-01-01 00:53:44|        null|        61.0|   null|                B00037|
|              B00037|2021-01-01 00:59:0

In [46]:
df.count()

1154112

In [47]:
df.head()

Row(dispatching_base_num='B00009', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 27), dropOff_datetime=datetime.datetime(2021, 1, 1, 0, 44), PUlocationID=None, DOlocationID=None, SR_Flag=None, Affiliated_base_number='B00009')

In [28]:
df.schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', TimestampNTZType(), True), StructField('dropOff_datetime', TimestampNTZType(), True), StructField('PUlocationID', DoubleType(), True), StructField('DOlocationID', DoubleType(), True), StructField('SR_Flag', IntegerType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [72]:
import pandas as pd 

df_pandas = pd.read_parquet('fhv_tripdata_2021-01.parquet')

In [73]:
df_pandas = df_pandas.head(1000)

In [74]:
df_pandas.dtypes

dispatching_base_num              object
pickup_datetime           datetime64[ns]
dropOff_datetime          datetime64[ns]
PUlocationID                     float64
DOlocationID                     float64
SR_Flag                           object
Affiliated_base_number            object
dtype: object

In [24]:
from pyspark.sql.types import *

# Define the schema based on the data types of the Pandas DataFrame
schema = StructType([
    StructField("dispatching_base_num", StringType(), nullable=True),
    StructField("pickup_datetime", TimestampType(), nullable=True),
    StructField("dropOff_datetime", TimestampType(), nullable=True),
    StructField("PUlocationID", DoubleType(), nullable=True),
    StructField("DOlocationID", DoubleType(), nullable=True),
    StructField("SR_Flag", StringType(), nullable=True),
    StructField("Affiliated_base_number", StringType(), nullable=True),
])

In [86]:
spark_df = spark.createDataFrame(df_pandas, schema)

In [87]:
spark_df.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B00009|2021-01-01 00:27:00|2021-01-01 00:44:00|         NaN|         NaN|   null|                B00009|
|              B00009|2021-01-01 00:50:00|2021-01-01 01:07:00|         NaN|         NaN|   null|                B00009|
|              B00013|2021-01-01 00:01:00|2021-01-01 01:51:00|         NaN|         NaN|   null|                B00013|
|              B00037|2021-01-01 00:13:09|2021-01-01 00:21:26|         NaN|        72.0|   null|                B00037|
|              B00037|2021-01-01 00:38:31|2021-01-01 00:53:44|         NaN|        61.0|   null|                B00037|
|              B00037|2021-01-01 00:59:0

In [88]:
spark_df.schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', TimestampType(), True), StructField('dropOff_datetime', TimestampType(), True), StructField('PUlocationID', DoubleType(), True), StructField('DOlocationID', DoubleType(), True), StructField('SR_Flag', StringType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [90]:
df = spark_df.repartition(24)

In [ ]:
df.write.parquet('fhv/2021/01/')

In [3]:
df = spark.read.parquet('fhv/2021/01/')

In [4]:
df

DataFrame[dispatching_base_num: string, pickup_datetime: timestamp, dropOff_datetime: timestamp, PUlocationID: double, DOlocationID: double, SR_Flag: string, Affiliated_base_number: string]

In [5]:
df.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B00310|2021-01-01 00:30:04|2021-01-01 00:46:57|         NaN|       169.0|   null|                B02875|
|              B00706|2021-01-01 00:23:43|2021-01-01 00:28:30|       221.0|       115.0|   null|                B00706|
|              B00706|2021-01-01 00:44:12|2021-01-01 00:51:08|       221.0|       206.0|   null|                B00706|
|              B00310|2021-01-01 00:39:37|2021-01-01 00:56:52|         NaN|       168.0|   null|                B02864|
|              B00256|2021-01-01 00:26:01|2021-01-01 00:30:42|         NaN|         NaN|   null|                B00256|
|              B00900|2021-01-01 00:49:0

In [7]:
df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropOff_datetime: timestamp (nullable = true)
 |-- PUlocationID: double (nullable = true)
 |-- DOlocationID: double (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [11]:
df.select('pickup_datetime','dropOff_datetime','PUlocationID','SR_Flag','dispatching_base_num') \
.filter(df.dispatching_base_num == "B00310") \
.show()

+-------------------+-------------------+------------+-------+--------------------+
|    pickup_datetime|   dropOff_datetime|PUlocationID|SR_Flag|dispatching_base_num|
+-------------------+-------------------+------------+-------+--------------------+
|2021-01-01 00:30:04|2021-01-01 00:46:57|         NaN|   null|              B00310|
|2021-01-01 00:39:37|2021-01-01 00:56:52|         NaN|   null|              B00310|
|2021-01-01 00:09:58|2021-01-01 00:23:24|         NaN|   null|              B00310|
|2021-01-01 00:45:08|2021-01-01 00:49:48|         NaN|   null|              B00310|
|2021-01-01 00:40:43|2021-01-01 00:49:15|         NaN|   null|              B00310|
|2021-01-01 00:10:00|2021-01-01 00:24:29|         NaN|   null|              B00310|
|2021-01-01 00:32:59|2021-01-01 00:52:49|         NaN|   null|              B00310|
|2021-01-01 00:59:08|2021-01-01 01:13:37|         NaN|   null|              B00310|
|2021-01-01 00:08:50|2021-01-01 00:21:46|         NaN|   null|              

SELECT * FROM df where dispatching_base_num = "B00310"

In [13]:
from pyspark.sql import functions as F 

In [19]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropOff_date', F.to_date(df.dropOff_datetime)) \
    .select('pickup_date','dropOff_date','PUlocationID','SR_Flag','dispatching_base_num') \
    .show()


+-----------+------------+------------+-------+--------------------+
|pickup_date|dropOff_date|PUlocationID|SR_Flag|dispatching_base_num|
+-----------+------------+------------+-------+--------------------+
| 2021-01-01|  2021-01-01|         NaN|   null|              B00310|
| 2021-01-01|  2021-01-01|       221.0|   null|              B00706|
| 2021-01-01|  2021-01-01|       221.0|   null|              B00706|
| 2021-01-01|  2021-01-01|         NaN|   null|              B00310|
| 2021-01-01|  2021-01-01|         NaN|   null|              B00256|
| 2021-01-01|  2021-01-01|         NaN|   null|              B00900|
| 2021-01-01|  2021-01-01|         NaN|   null|              B00856|
| 2021-01-01|  2021-01-01|         NaN|   null|              B00882|
| 2021-01-01|  2021-01-01|         NaN|   null|              B00856|
| 2021-01-01|  2021-01-01|       115.0|   null|              B00706|
| 2021-01-01|  2021-01-01|         NaN|   null|              B01145|
| 2021-01-01|  2021-01-01|        

In [21]:
def some_func(base_num):
    num = int(base_num[1:])

    if num % 7 == 0:
        return f's/{num:03x}'
    elif num % 3 == 0:
        return f'a/{num:03x}'
    else:
        return f'e/{num:03x}'

In [22]:
some_func('B01312')

'e/520'

In [25]:
some_func_udf = F.udf(some_func, returnType=StringType())

In [26]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropOff_date', F.to_date(df.dropOff_datetime)) \
    .withColumn('dispatching_base_num', some_func_udf(df.dispatching_base_num)) \
    .select('pickup_date','dropOff_date','PUlocationID','SR_Flag','dispatching_base_num') \
    .show()


+-----------+------------+------------+-------+--------------------+
|pickup_date|dropOff_date|PUlocationID|SR_Flag|dispatching_base_num|
+-----------+------------+------------+-------+--------------------+
| 2021-01-01|  2021-01-01|         NaN|   null|               e/136|
| 2021-01-01|  2021-01-01|       221.0|   null|               e/2c2|
| 2021-01-01|  2021-01-01|       221.0|   null|               e/2c2|
| 2021-01-01|  2021-01-01|         NaN|   null|               e/136|
| 2021-01-01|  2021-01-01|         NaN|   null|               e/100|
| 2021-01-01|  2021-01-01|         NaN|   null|               a/384|
| 2021-01-01|  2021-01-01|         NaN|   null|               e/358|
| 2021-01-01|  2021-01-01|         NaN|   null|               s/372|
| 2021-01-01|  2021-01-01|         NaN|   null|               e/358|
| 2021-01-01|  2021-01-01|       115.0|   null|               e/2c2|
| 2021-01-01|  2021-01-01|         NaN|   null|               e/479|
| 2021-01-01|  2021-01-01|        